### Spotify API

In [1]:
import requests
import json
import logging
import base64
import pymysql
import csv

import boto3
import os

### Connecting to Amazon RDS

In [2]:
## mysql -h testdb-spotify.c1ljsqo7birx.us-west-2.rds.amazonaws.com -P 3306 -D production -u admin -p
host="testdb-spotify.c1ljsqo7birx.us-west-2.rds.amazonaws.com"
port=3306
username="admin"
password="Wlqtnsdl11061310*"
database="production"

connection=pymysql.connect(host=host, user=username, password=password, database=database, port=port, use_unicode=True, charset="utf8")
cursor=connection.cursor()

cursor.execute("SHOW TABLES")
print(cursor.fetchall()) 

(('artist_genres',), ('artists',), ('people',))


### Connecting to Amazon DynamoDB

In [3]:
dynamodb = boto3.resource('dynamodb', region_name='us-west-2', endpoint_url="http://dynamodb.us-west-2.amazonaws.com")
table = dynamodb.Table('top_tracks')

### Access to Spotify API

In [4]:
client_id="b03942aa01eb477cacb26e9bf8d3da58"
response_type="code"
redirect_uri="http://localhost:8888"
#scope="user-read-private%20user-read-email"
endpoint1="https://accounts.spotify.com/authorize?client_id={}&response_type={}&redirect_uri={}".format(client_id, response_type, redirect_uri)
#endpoint1="https://accounts.spotify.com/authorize?client_id={}&response_type={}&redirect_uri={}&scope={}".format(client_id, response_type, redirect_uri, scope)
res1=requests.get(endpoint1)

print(res1.status_code)
print(res1.text)
print(endpoint1)

200

<!DOCTYPE html>
<html id="app" lang="en" dir="ltr" ng-csp ng-strict-di>
<head>
  <meta charset="utf-8">
  <title ng-bind="(&#39;loginTitle&#39; | localize) + &#39; - Spotify&#39;">Spotify</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
  <base href="/">
  <link rel="icon" href="https://accounts.scdn.co/sso/images/favicon.ace4d8543bbb017893402a1e9d1ac1fa.ico">
  <link href="https://accounts.scdn.co/sso/css/index.0da5aa431d52c8b0c32d.css" media="screen" rel="stylesheet">

  <script defer src="https://accounts.scdn.co/sso/js/index.0da5aa431d52c8b0c32d.js" sp-bootstrap></script>
  <meta ng-non-bindable sp-bootstrap-data='{"phoneFeatureEnabled":false,"previewEnabled":false,"user":false,"tpaState":"AQBsI+mM4+LcaPEQoJmcrGrMSLgRtNds3Ap53/NCGmnG8wHTSZglLpXGGx1L3lVKOCeYkeJrTqCi1MJsXMA+iEBsdsV0uXyq76qZO7SshWlzTshULn/1KssADrgZDnZmKsPpmLMy1Gt6HAOAAvlvQ9deLRhn/1SP5cIWakJhG2nRo1y1WlHd9DE0kN9VVqqWczYxneScF7Oh3C4+JEtnX858NeaCnyY

In [5]:
data={
    "grant_type":"authorization_code",
    "code":"AQAeqUxfNQDmDrlV_I8-COOA3Vv9oIm93wIp2nt1-IIExO4prbTX9f-eBGBPt2cX-2KeYmLdIiVGHyxfcnvlCmioIa71lIFWvo_tvEpbzUudoas_0Q-l8xEbqasjpK-lVBss9jfROvr7yluNvYQHPyMzXpzxDFdEug",
    "redirect_uri":"http://localhost:8888"
}


client_id="b03942aa01eb477cacb26e9bf8d3da58"
client_secret="fd61807827344135a98ec16305e34bb2"
encoded=base64.b64encode('{}:{}'.format(client_id, client_secret).encode('utf-8')).decode('ascii')
headers={"Authorization":"Basic {}".format(encoded)}

endpoint2="https://accounts.spotify.com/api/token"

res2=requests.post(endpoint2, data=data, headers=headers)

print(res2.status_code)
print(res2.text) #type=string

200
{"access_token":"BQBJImq_mo1P_tvHYTk84pK5JdAM-JWxdOz7vQ6hykipyT4obqLNw054l1WGu66K77Y9ByB3WcB7JJ8VDq_S7qWZjqqS0-xHlwf6EYrXbfayki20HaoOROTc9b9pEfW1KmX3ZPspUr3xtHidWjOmmYp3sRyTcfEm0HvB2w","token_type":"Bearer","expires_in":3600,"refresh_token":"AQCjtLj48bgF_QDlxbJzJHAm2M76tahQK1tZTqycLgIOKZjiyVS99C8Rgd0HvGmiHUMki4dsRRROMZ5myYUsjH-MgdvyBLXZuB3X12cSKE0pR3Bd1-ZSipvbG3SEtX0PXts"}


In [6]:
print(res2.json())
json.loads(res2.text) #type=dict

access_token=json.loads(res2.text)['access_token']
headers2={"Authorization":"Bearer {}".format(access_token)}

{'access_token': 'BQBJImq_mo1P_tvHYTk84pK5JdAM-JWxdOz7vQ6hykipyT4obqLNw054l1WGu66K77Y9ByB3WcB7JJ8VDq_S7qWZjqqS0-xHlwf6EYrXbfayki20HaoOROTc9b9pEfW1KmX3ZPspUr3xtHidWjOmmYp3sRyTcfEm0HvB2w', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQCjtLj48bgF_QDlxbJzJHAm2M76tahQK1tZTqycLgIOKZjiyVS99C8Rgd0HvGmiHUMki4dsRRROMZ5myYUsjH-MgdvyBLXZuB3X12cSKE0pR3Bd1-ZSipvbG3SEtX0PXts'}


### Get 'artist_id' from artists table in RDS and Retrieve data of top tracks

In [14]:
cursor.execute('SELECT id FROM artists LIMIT 3')
for (artist_id, ) in cursor.fetchall():
    URL = "https://api.spotify.com/v1/artists/{}/top-tracks".format(artist_id)
    params = {
        'country' : 'US'
    }
    
    
    r = requests.get(URL, params=params, headers=headers2)
    
    raw = json.loads(r.text)
    
    for track in raw['tracks']:
        data = {
            'artist_id' : artist_id
        }
        
        data.update(track)
        table.put_item(Item = data)

### If error "Unable to locate the credential" shows up,
#### - Run [\$ aws configure list] or [C:/> aws configure list] to see the summary of configure information
#### - Under the .aws/config, there should be default information of 'aws_access_key' 'aws_secret_access_key' 'region'
#### - If not, check the .aws/credentials. There should be default information of 'aws_access_key' and 'aws_secret_access_key'
#### - If .aws/credentials includes default information of 'aws_access_key' and 'aws_secret_access_key', run below commands
####   [set AWS_ACCESS_KEY = 'your aws_access_key']
####   [set AWS_SECRET_ACCESS_KEY = 'your aws_secret_access_key']

### Retrieve the data from DynamoDB

In [23]:
response = table.get_item(

Key ={
    "artist_id" : '02uYdhMhCgdB49hZlYRm9o',
    'id' : '1dlgaGdQXmsYKMRuewIZkF'
}   
)

print(response['Item'])


{'is_playable': True, 'duration_ms': Decimal('158600'), 'external_ids': {'isrc': 'US4N80500094'}, 'uri': 'spotify:track:1dlgaGdQXmsYKMRuewIZkF', 'name': 'Miss Alissa', 'album': {'total_tracks': Decimal('15'), 'images': [{'width': Decimal('640'), 'url': 'https://i.scdn.co/image/ab67616d0000b27384887df9c8e604955f804cda', 'height': Decimal('640')}, {'width': Decimal('300'), 'url': 'https://i.scdn.co/image/ab67616d00001e0284887df9c8e604955f804cda', 'height': Decimal('300')}, {'width': Decimal('64'), 'url': 'https://i.scdn.co/image/ab67616d0000485184887df9c8e604955f804cda', 'height': Decimal('64')}], 'artists': [{'name': 'Eagles Of Death Metal', 'href': 'https://api.spotify.com/v1/artists/02uYdhMhCgdB49hZlYRm9o', 'id': '02uYdhMhCgdB49hZlYRm9o', 'type': 'artist', 'external_urls': {'spotify': 'https://open.spotify.com/artist/02uYdhMhCgdB49hZlYRm9o'}, 'uri': 'spotify:artist:02uYdhMhCgdB49hZlYRm9o'}], 'release_date': '2004', 'name': 'Peace Love Death Metal', 'album_type': 'album', 'release_date

### How to Filter : Querying and Scanning
#### - Querying : use primary key
#### - Scanning : no need of primary key

In [24]:
from boto3.dynamodb.conditions import Key, Attr

In [29]:
response2 = table.query(
KeyConditionExpression=Key('artist_id').eq('02uYdhMhCgdB49hZlYRm9o'),
FilterExpression = Attr('popularity').gt(70)
)

print(response2) #no result is found

{'Items': [], 'Count': 0, 'ScannedCount': 10, 'ResponseMetadata': {'RequestId': 'MF81T1GMTDNFBIU9KKV52TSMVRVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'MF81T1GMTDNFBIU9KKV52TSMVRVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2380096782', 'content-type': 'application/x-amz-json-1.0', 'content-length': '40', 'date': 'Mon, 11 Oct 2021 18:23:59 GMT'}, 'RetryAttempts': 0}}


In [34]:
response3 = table.query(
KeyConditionExpression=Key('artist_id').eq('02uYdhMhCgdB49hZlYRm9o'),
FilterExpression = Attr('popularity').gt(50)
)

print(len(response3['Items']))

3


In [39]:
response4 = table.scan(
FilterExpression = Attr('popularity').gt(50)
)

print(len(response4['Items']))

23
